# Классификация тональности коротких текстов [TF-IDF]

In [1]:
import pandas as pd
import pymorphy2
import eli5
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from gensim.models import Phrases
from nltk.tokenize import word_tokenize
from tqdm import tqdm
tqdm.pandas()


### Работа с датасетом

In [2]:
# загрузить датасет
df = pd.read_csv(
    'women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')
df['sentiment'].value_counts()


negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [3]:
# убрать класс neutral
df = df[df['sentiment'] != 'neautral']


In [4]:
# убрать знаки препинания
df['review_processed'] = df['review'].apply(
    lambda x: re.sub(r'[^\w\s]', '', x)).values

# привести к нижнему регистру
df['review_processed'] = df['review_processed'].str.lower()


In [5]:
# токенизация
df['review_processed'] = df['review_processed'].progress_apply(
    lambda x: word_tokenize(x))


  0%|          | 0/60000 [00:00<?, ?it/s]

100%|██████████| 60000/60000 [00:07<00:00, 7754.31it/s]


In [6]:
# лемматизация
morph = pymorphy2.MorphAnalyzer()
df['review_lemmatized'] = df['review_processed'].progress_apply(
    lambda x: [morph.parse(word)[0].normal_form for word in x])


100%|██████████| 60000/60000 [03:07<00:00, 319.30it/s]


In [7]:
# генерация коллокаций
bigram = Phrases(df['review_lemmatized'].values, min_count=2, threshold=7)
df['review_lemmatized_coll'] = df['review_lemmatized'].progress_apply(
    lambda x: bigram[x])

# вывести найденные колокации
bigram.export_phrases()


100%|██████████| 60000/60000 [00:02<00:00, 28679.82it/s]


{'за_такой': 10.687130891423308,
 'другой_человек': 26.485106382978724,
 'ладный_хоть': 107.42918079405008,
 'деньга_вернуть': 8.553132410577003,
 'ничего_общий': 8.455688064859672,
 'на_картинка': 8.417227527576905,
 'сам_дело': 94.92785411846185,
 'продлить_защита': 151.28443495863405,
 'мой_согласие': 57.30904895772556,
 'голый_синтетик': 96.06008084671723,
 'понимать_почему': 88.75702990309732,
 'в_остальной': 9.049935185623038,
 'за_общительность': 12.163777089783283,
 'размер_s': 8.535167916211655,
 'тот_что': 10.09385892045114,
 'постоянно_сползать': 122.84210526315789,
 'капроновый_колготки': 298.9603842678461,
 'после_первый': 21.351067688588756,
 'первый_носка': 9.65976312448307,
 'сразу_же': 25.083938961791304,
 'до_граница': 14.538378991772847,
 'подозрение_что': 16.582768226455443,
 'продавец_отказываться': 8.512525582642933,
 'отказываться_возвращать': 157.54556722076407,
 'отправить_повторно': 36.29496762869614,
 'последний_раз': 8.909318234995201,
 'обещать_продлить': 1

In [15]:
# результат
df['review_lemmatized_coll'].iloc[0]


['качество',
 'плохой',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствовать',
 'ткань',
 'ужасный',
 'рисунок',
 'блёклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоить',
 'за_такой',
 'деньга',
 'г']

### Обучение логистической регрессии

In [9]:
# векторизация
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(
    df['review_lemmatized_coll'].apply(lambda x: ' '.join(x)))


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df['sentiment'], test_size=0.3, random_state=42)


In [11]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)


0.971785345911664

### GridSearchCV

In [12]:
# подбор гиперпараметров по сетке
import warnings
from sklearn.exceptions import ConvergenceWarning

# Игнорировать предупреждения о сходимости
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    parameters = {'C': [0.5, 0.75, 1, 5, 10, 15],
                  'max_iter': [50, 100, 200]}
    clf = GridSearchCV(logit, parameters)
    clf.fit(X_train, y_train)


In [13]:
clf.best_params_


{'C': 5, 'max_iter': 200}

In [14]:
logit = LogisticRegression(C=5, max_iter=200)
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)


0.9749518512222893